# Graph deep learning with keras 

I will discuss Graph deep learning in depth. I will focus in each kernel on one topic.
In this kernel, I will take screenshots from [Kipf paper](https://arxiv.org/pdf/1609.02907.pdf) and also from the [dgl library](https://github.com/dmlc/dgl/tree/master/examples/tensorflow/gcn)

## Motivation
> Many important real-world datasets come in the form of graphs or networks: social networks, knowledge graphs, protein-interaction networks, the World Wide Web, etc. (just to name a few). Yet, until recently, very little attention has been devoted to the generalization of neural network models to such structured datasets.
In the last couple of years, a number of papers re-visited this problem of generalizing neural networks to work on arbitrarily structured graphs (Bruna et al., ICLR 2014; Henaff et al., 2015; Duvenaud et al., NIPS 2015; Li et al., ICLR 2016; Defferrard et al., NIPS 2016; Kipf & Welling, ICLR 2017), some of them now achieving very promising results in domains that have previously been dominated by, e.g., kernel-based methods, graph-based regularization techniques and others.   WRITTEN BY [KIPF](https://tkipf.github.io/graph-convolutional-networks/) 


## GCN vs RGCN
This tutorial is for GCN. I wrote another [tutorial on RCGN.](https://www.kaggle.com/elmahy/relational-graph-convolution-networks-explained)

|  | <font style="color:red">Graph convolution networks</font> | <font style="color:red"> Relational Graph convolution networks</font> |
| - | - | - |
| Type of netwok that it works on | Simple just some nodes connected with edges | A directed labeled network  |
| Example | papers citing each others | knowledge graph (google it)
| Weights | Just make a dense layer for each node and pass messages from each node to the other | make a dense layer for each relation type since we have many types of relation e.g. ahmed employes(relation type) mohamed, John visits (relation type) [Dahab](https://www.youtube.com/watch?v=CheLNATs4IA&t=33s) because it's awesome and cheap and the author can invite you for dinner there.

# Fast approximate convolutions on graphs

GCN is just a layer similar to Dense, Conv2D,...layers. It has an activation function and some inputs, and output. This is the GCN formula.
![gcn.png](attachment:gcn.png)


![desc.png](attachment:desc.png) [reference](https://docs.dgl.ai/tutorials/models/1_gnn/1_gcn.html)

In the equation, there are A (adjacency matrix), squareroot of D (D to -1/2) , W and H.
1. Adjacency matrix is a binary matrix; It compares each node with all other nodes and if they are connected they score 1 else 0

![amatrix.png](attachment:amatrix.png)


2. Degree Matrix (D) is a diagonal matrix where each number refers to the number of degrees (edges; lines connected to) of each node. For example, here node(6) had only one edge while node(1) has 4 (Two other nodes plus two for itself (how many lines touch the circle around node (1) ? Four))

![degree_matrix.jpg](attachment:degree_matrix.jpg)


3. H(l) is the input which is a matrix of nodes as rows and each column is a feature e.g. if nodes represent users; features could include user age, country, etc. If there are no features we can the index number of each node as its sole feature. 

4. H(l+1) is computed from the equation above, and can be used as input again.

# Step 2 -> Decide which library to use 

I will use the dgl library (similar to keras) with tensorflow as backend

dgl is responsible for the  message passing  part and everything else is conducted by tensorflow

In [ ]:
import os
os.environ['DGLBACKEND'] = 'tensorflow'

!pip -q install dgl-cu101
!pip -q install tfdlpack-gpu  # when using tensorflow gpu version
!export TF_FORCE_GPU_ALLOW_GROWTH=true # and add this to your .bashrc/.zshrc file if needed

import dgl
print(dgl.backend.backend_name)



     |███████████████████████         | 25.9 MB 1.6 MB/s eta 0:00:07

# Step 3 -> Import everything you need

In [ ]:
# common imports
import time
import numpy as np
import networkx as nx
import requests

# tf part
import tensorflow as tf
from tensorflow.keras import layers

# dgl part
from dgl import DGLGraph
from dgl.data import register_data_args, load_data

# Step 4 -> Load the data

For a very secret reason, I will not say why I loaded the data this way. 

Here I loaded the cora dataset. (Google it for more information )

You can try with any of the three datasets available from dgl
 (available dataset: "cora", "citeseer", "pubmed")
## Cora dataset


## Citeseer dataset


    CiteSeer for Document Classification
        The CiteSeer dataset consists of 3312 scientific publications classified into one of six classes. The citation network consists of 4732 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 3703 unique words. The README file in the dataset provides more details.
        Download link:
            https://linqs-data.soe.ucsc.edu/public/lbc/citeseer.tgz  
        Related papers:
            Qing Lu, and Lise Getoor. "Link-based classification." ICML, 2003.
            Prithviraj Sen, et al. "Collective classification in network data." AI Magazine, 2008.  source -> https://linqs.soe.ucsc.edu/data


## Pubmed dataset

    PubMed Diabetes
        The Pubmed Diabetes dataset consists of 19717 scientific publications from PubMed database pertaining to diabetes classified into one of three classes. The citation network consists of 44338 links. Each publication in the dataset is described by a TF/IDF weighted word vector from a dictionary which consists of 500 unique words. The README file in the dataset provides more details.
        Download Link:
            https://linqs-data.soe.ucsc.edu/public/Pubmed-Diabetes.tgz
        Related Papers:
            Galileo Namata, et. al. "Query-driven Active Surveying for Collective Classification." MLG. 2012. source -> https://linqs.soe.ucsc.edu/data


In [ ]:
class available_datasets_already_available_with_dgl:
    def __init__(self, dataset_name):
        self.dataset = dataset_name

data = load_data(available_datasets_already_available_with_dgl('citeseer'))

# Step 5 -> Extract needed information from the dataset

In [ ]:
features = tf.convert_to_tensor(data.features, dtype=tf.float32)
labels = tf.convert_to_tensor(data.labels, dtype=tf.int64)
train_mask = tf.convert_to_tensor(data.train_mask, dtype=tf.bool)
val_mask = tf.convert_to_tensor(data.val_mask, dtype=tf.bool)
test_mask = tf.convert_to_tensor(data.test_mask, dtype=tf.bool)

in_feats = features.shape[1]
n_classes = data.num_labels
n_edges = data.graph.number_of_edges()


print("""----Data statistics------'
#Edges %d
#Classes %d
#Train samples %d
#Val samples %d
#Test samples %d""" %
      (n_edges, n_classes,
       train_mask.numpy().sum(),
       val_mask.numpy().sum(),
       test_mask.numpy().sum()))

# Step 6 -> Extract the graph from the data and put it into a DGL graph 

DGL graph is just a simple graph, but it works well with tensorflow.

In [ ]:
# graph preprocess and calculate normalization factor
g = data.graph
g.remove_edges_from(nx.selfloop_edges(g))
g.add_edges_from(zip(g.nodes(), g.nodes()))

g = DGLGraph(g)


# nx.draw(g.to_networkx())  # you can draw it better than me. I am useless

# Step 7 -> Do some normalization on the graph

Connect each node to itself ... why ? I assume that if I am affected by others in the network, I must be affected by myself in the first place, huh ?

Do some square root stuff... just normalization .... not a big deal ... like scaling columns we used to do with pandas dataframes...huh ?

In [ ]:

n_edges = g.number_of_edges()

degs = tf.cast(tf.identity(g.in_degrees()), dtype=tf.float32)
norm = tf.math.pow(degs, -0.5)
norm = tf.where(tf.math.is_inf(norm), tf.zeros_like(norm), norm)
g.ndata['norm'] = tf.expand_dims(norm, -1)


# Step 8 -> Define the layers you will need

Regardless of what I am going to do , I will need to define my own layers (Although the GraphConv layer is predefined in dgl, I prefered to write it on my own).
Ignore normalization and baises for now.

To simplify,

1. Remember the most simple classification problem you ever learnt.
2. You have some users for each user you know [age, gender, height, weight, education, etc] and you want to predict his salary.
3. Another example, Imaging some users on twitter writing tweets and you would like to predict if they are happy or not.

4. Imagine the these users are connected in a network
5. Forget about the network and think about the users
6. Remember the very simple dense layer that you always do for any classification problem 
7. You will have input_features (age, gender, height,education) and output_features (salary)
8. Make a simple dense layer that take input_features of each node  and output the output_features

All the above is the same for any network

9. But, 
10. But, this is a network
11. In short, exchange the features between the nodes
Calculate the output features of each node, and, and, and, give each node the summation of the output features of all nodes around it.

        graph.ndata['h'] = feat
        graph.update_all(dgl.function.copy_src(src='h', out='m'),
                             dgl.function.sum(msg='m', out='h'))
        rst = graph.ndata['h']



In [ ]:
class GraphConv(layers.Layer):
    def __init__(self,
                out_feats):
        super(GraphConv, self).__init__()
        
        # a very simple dense layer takes node features as input and outputs a lower dimension representation
        self.denselayer = layers.Dense(out_feats, use_bias=False)

    def call(self, graph, feat):
        # make a local copy of the graph -- something related to not changing the global variable graph, I dont really care...
        graph = graph.local_var()

        feat = self.denselayer(feat)
        graph.ndata['h'] = feat
        graph.update_all(dgl.function.copy_src(src='h', out='m'),
                         dgl.function.sum(msg='m', out='h'))
        rst = graph.ndata['h']

        return rst


# Step 9 -> describe the GCN (Graph convolution network) model

Similar to any linear model that we write as a class

you will define the layers in the init method and use them in the call method.

In [ ]:
class GCN(tf.keras.Model):
    def __init__(self, g,in_feats,n_hidden,n_classes):
        super(GCN, self).__init__()
        self.g = g
        
        self.input_layer = GraphConv(n_hidden)
        self.hidden_layer = GraphConv(n_hidden)  # create more if you like
        self.output_layer = GraphConv(n_classes)
    
    def call(self, features):
        h = features
        h = self.input_layer(self.g, h)
        h = self.hidden_layer(self.g, h)
        h = self.output_layer(self.g, h)
        return h
        

# Step 10 -> Build the model

In [ ]:
# create GCN model

model = GCN(g,
           in_feats = in_feats,
           n_hidden=16,
           n_classes=n_classes)

loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2, epsilon=1e-8)


# Step 11 -> Train the model

In [ ]:
# put it in a loop for ten times you will get 79% accuracy. WoW~!
for x in range (100):
    with tf.GradientTape() as tape:
        logits = model(features)
        loss_value = loss_func(labels[train_mask], logits[train_mask])

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))



    # evaluate
    logits_test = model(features, training = False)

    logits_val = logits[val_mask]
    labels_val = labels[val_mask]
    indices = tf.math.argmax(logits_val, axis = 1)
    acc = tf.reduce_mean(tf.cast(indices == labels_val, dtype = tf.float32))

    print(acc.numpy().round(2))

# Step 12 -> Learn more about the model

In [ ]:
model.summary()